<a href="https://colab.research.google.com/github/r-ap/TrainingMachineLearning/blob/main/BFI_PYIM_15_%26_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Boosting

## Feature Engineering

Data dan proses yang sama seperti sesi 13

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

df = pd.read_csv('https://raw.githubusercontent.com/ardhiraka/FSDS_Guidelines/master/p1/v3/w1/P1W1D4AM%20-%20KNN%20%26%20Naive%20Bayes.csv')


X = df.drop(columns='Outcome')
y = df['Outcome']

X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.8, stratify=y)

y_train_0 = y_train[y_train==0]
y_train_1 = y_train[y_train==1]

y_train_0_sample = y_train_0.sample(len(y_train_1))
y_train_sample = pd.concat([y_train_0_sample,y_train_1]).sample(2*len(y_train_1))
y_train_sample

X_train_sample = X_train.loc[y_train_sample.index]

X_train_samp = X_train_sample[(X_train_sample['BMI']>0) & (X_train_sample['BloodPressure']>0) & (X_train_sample['Glucose']>0)]
y_train_samp = y_train_sample.loc[X_train_samp.index]

### AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

ADB = AdaBoostClassifier(n_estimators=50)

ADB.fit(X_train_samp,y_train_samp)

y_pred_train_adb = ADB.predict(X_train_samp)
y_pred_test_adb = ADB.predict(X_test)

print("Training")
print(classification_report(y_train_samp,y_pred_train_adb))

print("\n===========================\n")
print("Test")
print(classification_report(y_test,y_pred_test_adb))


Training
              precision    recall  f1-score   support

           0       0.86      0.84      0.85       206
           1       0.84      0.85      0.85       198

    accuracy                           0.85       404
   macro avg       0.85      0.85      0.85       404
weighted avg       0.85      0.85      0.85       404



Test
              precision    recall  f1-score   support

           0       0.81      0.73      0.77       100
           1       0.58      0.69      0.63        54

    accuracy                           0.71       154
   macro avg       0.69      0.71      0.70       154
weighted avg       0.73      0.71      0.72       154



### Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

GDB = GradientBoostingClassifier(n_estimators=50, criterion='squared_error', min_samples_split=3, min_samples_leaf=2)

GDB.fit(X_train_samp,y_train_samp)

y_pred_train_gdb = GDB.predict(X_train_samp)
y_pred_test_gdb = GDB.predict(X_test)

print("Training")
print(classification_report(y_train_samp,y_pred_train_gdb))

print("\n===========================\n")
print("Test")
print(classification_report(y_test,y_pred_test_gdb))


Training
              precision    recall  f1-score   support

           0       0.90      0.88      0.89       206
           1       0.88      0.89      0.89       198

    accuracy                           0.89       404
   macro avg       0.89      0.89      0.89       404
weighted avg       0.89      0.89      0.89       404



Test
              precision    recall  f1-score   support

           0       0.79      0.73      0.76       100
           1       0.56      0.65      0.60        54

    accuracy                           0.70       154
   macro avg       0.68      0.69      0.68       154
weighted avg       0.71      0.70      0.71       154



# Hyperparameter Tunning

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X_train_samp)
X_train_scaled = scaler.transform(X_train_samp)
X_test_scaled = scaler.transform(X_test)

params = {'kernel':['rbf','poly','sigmoid'],
          'C':[0.1,1,10]}

grid = GridSearchCV(estimator=SVC(),
                    param_grid=params,
                    cv=5,
                    verbose=3,
                    scoring='accuracy')

grid.fit(X_train_scaled,y_train_samp)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5] END .................C=0.1, kernel=rbf;, score=0.716 total time=   0.0s
[CV 2/5] END .................C=0.1, kernel=rbf;, score=0.716 total time=   0.0s
[CV 3/5] END .................C=0.1, kernel=rbf;, score=0.753 total time=   0.0s
[CV 4/5] END .................C=0.1, kernel=rbf;, score=0.691 total time=   0.0s
[CV 5/5] END .................C=0.1, kernel=rbf;, score=0.662 total time=   0.0s
[CV 1/5] END ................C=0.1, kernel=poly;, score=0.728 total time=   0.0s
[CV 2/5] END ................C=0.1, kernel=poly;, score=0.728 total time=   0.0s
[CV 3/5] END ................C=0.1, kernel=poly;, score=0.778 total time=   0.0s
[CV 4/5] END ................C=0.1, kernel=poly;, score=0.741 total time=   0.0s
[CV 5/5] END ................C=0.1, kernel=poly;, score=0.675 total time=   0.0s
[CV 1/5] END .............C=0.1, kernel=sigmoid;, score=0.358 total time=   0.0s
[CV 2/5] END .............C=0.1, kernel=sigmoid;,

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [0.1, 1, 10],
                         'kernel': ['rbf', 'poly', 'sigmoid']},
             scoring='accuracy', verbose=3)

In [ ]:
grid.best_params_

{'C': 1, 'kernel': 'rbf'}

In [ ]:
grid.best_score_

0.7325925925925926

In [ ]:
y_pred_train_grid = grid.predict(X_train_scaled)
y_pred_test_grid = grid.predict(X_test_scaled)

print("Training")
print(classification_report(y_train_samp,y_pred_train_grid))

print("\n===========================\n")
print("Test")
print(classification_report(y_test,y_pred_test_grid))

Training
              precision    recall  f1-score   support

           0       0.80      0.77      0.78       206
           1       0.77      0.79      0.78       198

    accuracy                           0.78       404
   macro avg       0.78      0.78      0.78       404
weighted avg       0.78      0.78      0.78       404



Test
              precision    recall  f1-score   support

           0       0.84      0.77      0.80       100
           1       0.63      0.72      0.67        54

    accuracy                           0.75       154
   macro avg       0.73      0.75      0.74       154
weighted avg       0.76      0.75      0.76       154



# Deployment

In [ ]:
import pickle

pickle.dump(scaler, open('scaler.pkl', 'wb')) #save scaler
pickle.dump(grid, open('model_grid.pkl', 'wb')) #save model grid

In [ ]:
loaded_grid = pickle.load(open('model_grid.pkl','rb'))
loaded_grid

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [0.1, 1, 10],
                         'kernel': ['rbf', 'poly', 'sigmoid']},
             scoring='accuracy', verbose=3)

In [ ]:
loaded_grid.best_params_

{'C': 1, 'kernel': 'rbf'}

## Flask app

```
Deployment/
    ├── model/
    │   └── model.pkl
    |   └── scaler.pkl
    ├── static
    │   └── css
    │        └── style.css
    ├── templates/
    │   └── main.html
    └── app.py
```

### app.py

```
import numpy as np
import pickle
from flask import Flask, render_template, request

app = Flask(__name__, template_folder='templates')

scaler = pickle.load(open('model/scaler.pkl', 'rb'))
model = pickle.load(open('model/model_grid.pkl', 'rb'))

@app.route('/')
def main():
    return(render_template('main.html'))

@app.route('/',methods=['POST'])
def predict():
    '''
    For rendering results on HTML GUI
    '''
    features = np.array([[float(x) for x in request.form.values()]])
    scaled_features = scaler.transform(features)
    prediction = model.predict(scaled_features)

    output = {0: 'not diabetes', 1: 'diabetes'}

    return render_template('main.html', features=features, prediction_text='Patient must be {}'.format(output[prediction[0]]))

if __name__ == '__main__':
    app.run()
```

### style.css

```
@import url(https://fonts.googleapis.com/css?family=Roboto:300);

.form {
  position: relative;
  z-index: 1;
  background: #FFFFFF;
  max-width: 360px;
  margin: 0 auto 100px;
  padding: 45px;
  text-align: center;
  box-shadow: 0 0 20px 0 rgba(0, 0, 0, 0.2), 0 5px 5px 0 rgba(0, 0, 0, 0.24);
}
.form input {
  font-family: "Roboto", sans-serif;
  outline: 0;
  background: #f2f2f2;
  width: 100%;
  border: 0;
  margin: 0 0 15px;
  padding: 15px;
  box-sizing: border-box;
  font-size: 14px;
}
.form button {
  font-family: "Roboto", sans-serif;
  text-transform: uppercase;
  outline: 0;
  background: #F19023;
  width: 100%;
  border: 0;
  padding: 15px;
  color: #FFFFFF;
  font-size: 14px;
  cursor: pointer;
}
.form button:hover,.form button:active,.form button:focus {
  background: #c5761c;
}
.form .message {
  margin: 15px 0 0;
  color: #b3b3b3;
  font-size: 12px;
}

body {
  background: #F19023;
  font-family: "Roboto", sans-serif;  
}
```

```
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Model Deployment</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='css/style.css') }}">
</head>
<body>

    <div class="form">
        <h1>Diabetes Patient Classifier</h1>

        <form action="{{ url_for('predict') }}" method="post">
            <input type="text" name="preg" placeholder="Pregnancies" required="required" />
            <input type="text" name="gluc" placeholder="Glucose" required="required" />
            <input type="text" name="blodpre" placeholder="Blood Presure" required="required" />
            <input type="text" name="skin" placeholder="Skin Thickness" required="required" />
            <input type="text" name="insul" placeholder="Insulin" required="required" />
            <input type="text" name="bmi" placeholder="BMI" required="required" />
            <input type="text" name="dpf" placeholder="Diabetes Pedigree Function" required="required" />
            <input type="text" name="age" placeholder="Age" required="required" />

            <button type="submit" class="btn btn-primary btn-block btn-large">Predict</button>
        </form>

        {% if features|length>0 %}
            <h1 class="message">{{ prediction_text }}</h1>
        {% endif %}
    </div>

</body>
</html>
```